In [124]:

# Optional: Create the CSV file using Python
content = """
transaction_id,price,quantity
T5001,1500,2
T5002,500,10
T5003,12000,1
T5004,1500,5
T5005,12000,3
"""

with open("transactions.csv", "w") as file:
    file.write(content)

print("transactions.csv created successfully!")


transactions.csv created successfully!


In [125]:
import pandas as pd
import numpy as np

df = pd.read_csv("transactions.csv")
print("=== LOADED DATASET ===")
print(df)
print(f"\nShape: {df.shape}")


=== LOADED DATASET ===
  transaction_id  price  quantity
0          T5001   1500         2
1          T5002    500        10
2          T5003  12000         1
3          T5004   1500         5
4          T5005  12000         3

Shape: (5, 3)


In [126]:
print("\n=== DATA TYPES ===")
print(df.dtypes)



=== DATA TYPES ===
transaction_id    object
price              int64
quantity           int64
dtype: object


In [127]:
prices = df["price"].values
quantities = df["quantity"].values

print("=== NUMPY ARRAYS ===")
print(f"Prices array: {prices}")
print(f"Quantities array: {quantities}")
print(f"Prices dtype: {prices.dtype}")
print(f"Quantities dtype: {quantities.dtype}")


=== NUMPY ARRAYS ===
Prices array: [ 1500   500 12000  1500 12000]
Quantities array: [ 2 10  1  5  3]
Prices dtype: int64
Quantities dtype: int64


In [128]:
print(f"\nPrices is NumPy array: {isinstance(prices, np.ndarray)}")
print(f"Quantities is NumPy array: {isinstance(quantities, np.ndarray)}")



Prices is NumPy array: True
Quantities is NumPy array: True


In [129]:
revenue = prices * quantities
print("=== REVENUE CALCULATION ===")
print("Revenue:", revenue)
print("No loops used! ✓")


=== REVENUE CALCULATION ===
Revenue: [ 3000  5000 12000  7500 36000]
No loops used! ✓


In [130]:
df["revenue"] = revenue
print("\n=== DATAFRAME WITH REVENUE ===")
print(df)



=== DATAFRAME WITH REVENUE ===
  transaction_id  price  quantity  revenue
0          T5001   1500         2     3000
1          T5002    500        10     5000
2          T5003  12000         1    12000
3          T5004   1500         5     7500
4          T5005  12000         3    36000


In [131]:
categories = np.where(
    revenue > 10000,
    "HIGH",
    np.where(
        revenue >= 2000,
        "MEDIUM",
        "LOW"
    )
)

print("=== CLASSIFICATION ===")
print("Categories:", categories)


=== CLASSIFICATION ===
Categories: ['MEDIUM' 'MEDIUM' 'HIGH' 'MEDIUM' 'HIGH']


In [132]:
print("\n=== CLASSIFICATION VERIFICATION ===")
for i in range(len(revenue)):
    print(f"Transaction {df.iloc[i]['transaction_id']}: "
          f"Revenue={revenue[i]:,} → Category={categories[i]}")



=== CLASSIFICATION VERIFICATION ===
Transaction T5001: Revenue=3,000 → Category=MEDIUM
Transaction T5002: Revenue=5,000 → Category=MEDIUM
Transaction T5003: Revenue=12,000 → Category=HIGH
Transaction T5004: Revenue=7,500 → Category=MEDIUM
Transaction T5005: Revenue=36,000 → Category=HIGH


In [133]:
df["category"] = categories
print("\n=== FINAL DATAFRAME ===")
print(df)



=== FINAL DATAFRAME ===
  transaction_id  price  quantity  revenue category
0          T5001   1500         2     3000   MEDIUM
1          T5002    500        10     5000   MEDIUM
2          T5003  12000         1    12000     HIGH
3          T5004   1500         5     7500   MEDIUM
4          T5005  12000         3    36000     HIGH


In [134]:
output_df = df[["transaction_id", "revenue", "category"]].copy()
print("=== OUTPUT DATAFRAME ===")
print(output_df)


=== OUTPUT DATAFRAME ===
  transaction_id  revenue category
0          T5001     3000   MEDIUM
1          T5002     5000   MEDIUM
2          T5003    12000     HIGH
3          T5004     7500   MEDIUM
4          T5005    36000     HIGH


In [135]:
output_df.to_csv("transaction_classification.csv", index=False)
print("\nSaved: transaction_classification.csv")



Saved: transaction_classification.csv


In [136]:
print("\n=== SUMMARY STATISTICS ===")
print(f"Total transactions: {len(output_df)}")
print(f"Total revenue: {output_df['revenue'].sum():,.2f}")
print(f"Average revenue: {output_df['revenue'].mean():,.2f}")
print(f"\nCategory distribution:")
print(output_df["category"].value_counts())



=== SUMMARY STATISTICS ===
Total transactions: 5
Total revenue: 63,500.00
Average revenue: 12,700.00

Category distribution:
category
MEDIUM    3
HIGH      2
Name: count, dtype: int64


In [137]:
def business_rules_engine(input_file, output_file):
    """
    Apply business rules to transactions using NumPy vectorization.
    
    Args:
        input_file (str): Path to input CSV file
        output_file (str): Path to output CSV file
    
    Returns:
        pd.DataFrame: Classified transactions
    """
    # Load data
    df = pd.read_csv(input_file)
    
    # Convert to NumPy arrays
    prices = df["price"].values
    quantities = df["quantity"].values
    
    # Compute revenue (vectorized)
    revenue = prices * quantities
    
    # Apply classification rules (vectorized)
    categories = np.where(
        revenue > 10000,
        "HIGH",
        np.where(
            revenue >= 2000,
            "MEDIUM",
            "LOW"
        )
    )
    
    # Create output DataFrame
    output_df = pd.DataFrame({
        "transaction_id": df["transaction_id"].values,
        "revenue": revenue,
        "category": categories
    })
    
    # Save output
    output_df.to_csv(output_file, index=False)
    
    print("Business rules engine executed successfully!")
    print(f"  - Processed {len(output_df)} transactions")
    print(f"  - Total revenue: {revenue.sum():,.2f}")
    print(f"  - Output saved to: {output_file}")
    
    return output_df


In [138]:
result = business_rules_engine("transactions.csv", "transaction_classification.csv")


Business rules engine executed successfully!
  - Processed 5 transactions
  - Total revenue: 63,500.00
  - Output saved to: transaction_classification.csv


In [139]:
# Read the function source to verify
import inspect
source = inspect.getsource(business_rules_engine)
print("=== CODE INSPECTION ===")
print("Checking for loops in classification logic...")

if "for" in source and "category" in source:
    # Check if 'for' is in a comment or actual loop
    lines = source.split('\n')
    for i, line in enumerate(lines):
        if 'for' in line and 'category' in source[max(0, i-5):i+5]:
            if not line.strip().startswith('#'):
                print(f"⚠️ Potential loop found: {line.strip()}")
else:
    print("✓ No loops detected in classification logic")


=== CODE INSPECTION ===
Checking for loops in classification logic...
✓ No loops detected in classification logic


In [140]:
print("\n=== VECTORIZATION APPROACH ===")
print("✓ Revenue calculation: prices * quantities (vectorized)")
print("✓ Classification: np.where() with nested conditions (vectorized)")
print("✓ All operations use NumPy array operations")
print("✓ No Python loops for business logic")



=== VECTORIZATION APPROACH ===
✓ Revenue calculation: prices * quantities (vectorized)
✓ Classification: np.where() with nested conditions (vectorized)
✓ All operations use NumPy array operations
✓ No Python loops for business logic
